In [92]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack

from sklearn.linear_model import SGDClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot

In [2]:
dfs = []

for df in pd.read_csv('train', chunksize=100000):
    df_ = pd.concat([df.loc[df['click'] == 0],
                     df.loc[df['click'] == 1].sample(frac=2, replace=True)])
    
    dfs.append(df_.sample(frac=0.2))
    
df = pd.concat(dfs)
del dfs

In [3]:
df.to_csv('train-subset.csv', index=False)

In [6]:
df.nunique()

id                  8975510
click                     2
hour                    240
C1                        7
banner_pos                7
site_id                3925
site_domain            5265
site_category            24
app_id                 6135
app_domain              373
app_category             30
device_id            947768
device_ip           2871562
device_model           7028
device_type               5
device_conn_type          4
C14                    2518
C15                       8
C16                       9
C17                     431
C18                       4
C19                      68
C20                     170
C21                      60
dtype: int64

In [78]:
d = {'C1':10, 'C14':3000, 'C15':10, 'C16':10, 'C17':500,
     'C18':5, 'C19':70, 'C20':180, 'C21':65}

X = np.hstack([to_categorical(df[col].head(10000).apply(lambda r: (one_hot(str(r), n_features))).tolist(), n_features)
               for (col, n_features) in d.items()])

X = np.hstack([X, df['banner_pos'].head(10000).values.reshape(-1, 1)])          

y = df['click'].head(10000).values

In [87]:
clf = SGDClassifier(loss='log',
                    verbose=1,
                    max_iter=10)

In [89]:
clf.fit(X, y)

-- Epoch 1
Norm: 24.18, NNZs: 499, Bias: -2.572278, T: 10000, Avg. loss: 4.150928
Total training time: 0.09 seconds.
-- Epoch 2
Norm: 16.43, NNZs: 499, Bias: -2.507175, T: 20000, Avg. loss: 1.184114
Total training time: 0.18 seconds.
-- Epoch 3
Norm: 13.77, NNZs: 499, Bias: -1.595455, T: 30000, Avg. loss: 0.843466
Total training time: 0.28 seconds.
-- Epoch 4
Norm: 12.46, NNZs: 499, Bias: -1.597841, T: 40000, Avg. loss: 0.726650
Total training time: 0.37 seconds.
-- Epoch 5
Norm: 11.59, NNZs: 499, Bias: -1.228064, T: 50000, Avg. loss: 0.669071
Total training time: 0.46 seconds.
-- Epoch 6
Norm: 11.10, NNZs: 499, Bias: -1.262151, T: 60000, Avg. loss: 0.633282
Total training time: 0.56 seconds.
-- Epoch 7
Norm: 10.77, NNZs: 499, Bias: -0.949787, T: 70000, Avg. loss: 0.620607
Total training time: 0.65 seconds.
-- Epoch 8
Norm: 10.73, NNZs: 499, Bias: -1.100357, T: 80000, Avg. loss: 0.603514
Total training time: 0.76 seconds.
-- Epoch 9
Norm: 10.46, NNZs: 499, Bias: -1.195102, T: 90000, Av

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=10, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=1, warm_start=False)

In [109]:
model = Sequential() 
model.add(Dense(units=2,
                input_dim=X.shape[1],
                activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['categorical_accuracy'])

In [111]:
model.fit(x=X, y=to_categorical(y), epochs=4)

Epoch 1/4
10000/10000 [==============================] - 1s 56us/step - loss: 0.5745 - categorical_accuracy: 0.7060
Epoch 2/4
10000/10000 [==============================] - 1s 56us/step - loss: 0.5655 - categorical_accuracy: 0.7286
Epoch 3/4
10000/10000 [==============================] - 1s 56us/step - loss: 0.5609 - categorical_accuracy: 0.7316
Epoch 4/4
10000/10000 [==============================] - 1s 55us/step - loss: 0.5582 - categorical_accuracy: 0.7325
